In [ ]:
# Copyright 2023 The Cirq Developers
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# And

To do classical logic with a reversible circuit (a pre-requisite for a quantum circuit), we use a three (qu)bit operation called a Toffoli gate that takes `[a, b, c]` to `[a, b, c ^ (a & b)]`. If we take `c` to be zero, this is an And gate taking `[a, b]` to `[a, b, a & b]`.

In [ ]:
import itertools
for a, b, in itertools.product([0, 1], repeat=2):
    print(a, b, '->', a & b)

## Quantum operation

We provide a quantum operation for performing quantum And. Specifically, it assumes the third qubit (i.e. the target) is initialized to the `|0>` state.

In [ ]:
import cirq
from cirq.contrib.svg import SVGCircuit
from cirq_ft import And

gate = And()
r = gate.registers
quregs = r.get_named_qubits()
operation = gate.on_registers(**quregs)
circuit = cirq.Circuit(operation)
SVGCircuit(circuit)

## Efficient decomposition

This specialization of the Toffoli gate permits a specialized decomposition that minimizes the `T`-gate count.

In [ ]:
c2 = cirq.Circuit(cirq.decompose_once(operation))
SVGCircuit(c2)

## Test behavior

We can test the behavior of the And gate on computational basis states using a Quantum simulator.

In [ ]:
import numpy as np

input_states = [(a, b, 0) for a, b in itertools.product([0, 1], repeat=2)]
output_states = [(a, b, a & b) for a, b, _ in input_states]


for inp, out in zip(input_states, output_states):
    result = cirq.Simulator(dtype=np.complex128).simulate(c2, initial_state=inp)
    print(inp, '->', result.dirac_notation())
    assert result.dirac_notation()[1:-1] == "".join(str(x) for x in out)

In [ ]:
inds, = np.where(abs(result.final_state_vector) > 1e-8)
assert len(inds) == 1
ind, = inds
f'{ind:3b}'

## Uncompute

We can save even more `T` gates when "uncomputing" an And operation, i.e. performing the adjoint operation by using classical control.

In [ ]:
inv_operation = operation ** -1
inv_circuit = cirq.Circuit(inv_operation)
SVGCircuit(inv_circuit)

We reset our target using measurement and fix up phases depending on the result of that measurement:

In [ ]:
inv_c2 = cirq.Circuit(cirq.decompose_once(inv_operation))
inv_c2

## Test Adjoint

In [ ]:
input_states = [(a, b, a & b) for a, b in itertools.product([0, 1], repeat=2)]
output_states = [(a, b, 0) for a, b, _ in input_states]

for inp, out in zip(input_states, output_states):
    result = cirq.Simulator().simulate(inv_circuit, initial_state=inp)
    print(inp, '->', result.dirac_notation())
    assert result.dirac_notation()[1:-1] == "".join(str(x) for x in out)